In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Updating the file path to match the user's directory structure: "data/csv/tmp/face_landmarks_20241016_212540.csv"
file_path = 'data/20times_4points_gum/participant/kojima/face_landmarks_20241217_112044_690615.csv'

# Load the CSV file from the updated directory (hypothetical)
df = pd.read_csv(file_path)

# Filter the data
df = df[df['Timestamp'] < (df['Timestamp'].max() * 7 // 8)]
df = df[df['Timestamp'] > (df['Timestamp'].max() * 1 // 8)]

# Extract the Frame and Distance columns for plotting
frames = df['Timestamp']
distances = df['Distance']

# Create a plot with Timestamp on the x-axis and Distance on the y-axis
plt.figure(figsize=(20, 6))
plt.plot(frames, distances, marker='o', linestyle='-', color='b')
plt.title('Timestamp vs Distance (All Frames)')
plt.xlabel('Timestamp')
plt.ylabel('Distance')
plt.grid(True)

# Display the plot
plt.show()

In [4]:
import csv
from scipy.stats import zscore

# CSVファイルのパスを指定してください
csv_filename = 'data/20times_4points_gum/participant/kojima/face_landmarks_20241217_112044_690615.csv'

# ピーク検出のパラメータ
Z_THRESHOLD = 1.1    # Zスコアの閾値
MIN_INTERVAL = 0    # ピーク間の最小フレーム間隔

distances = []
timestamps = []
frame_numbers = []

# データ全体を読み込み
all_data = []
with open(csv_filename, 'r', newline='') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    for row in csv_reader:
        all_data.append({
            'Distance': float(row['Distance']),
            'Timestamp': int(row['Timestamp']),
            'Frame_Count': int(row['Frame_Count'])
        })

# タイムスタンプでフィルタリング
max_timestamp = max(d['Timestamp'] for d in all_data)
filtered_data = [
    row for row in all_data
    if max_timestamp * 0 // 8 < row['Timestamp'] < max_timestamp * 8 // 8
]

# フィルタリングされたデータで処理を再開
peak_timestamps = []
interval_count = 0
flag = False

for row in filtered_data:
    distance = row['Distance']
    timestamp = row['Timestamp']
    frame_number = row['Frame_Count']

    distances.append(distance)
    timestamps.append(timestamp)
    frame_numbers.append(frame_number)

    # 直近30件のデータでのピーク検出
    recent_distances = distances[-30:]
    z_scores = zscore(recent_distances)
    
    mean_distances = sum(recent_distances) / len(recent_distances)

    if interval_count > 0:
        interval_count -= 1
    elif abs(z_scores[-1]) > Z_THRESHOLD and (mean_distances < recent_distances[0] and not(flag)):
        peak_timestamps.append(timestamp)
        interval_count = MIN_INTERVAL
        flag = True
    elif abs(z_scores[-1]) > Z_THRESHOLD and (mean_distances > recent_distances[0] and flag):
        flag = False

total_peaks = len(peak_timestamps)

print(f'ピークの総数: {total_peaks}')
print('ピークが検出されたタイムスタンプ:')
for ts in peak_timestamps:
    print(ts)

ピークの総数: 22
ピークが検出されたタイムスタンプ:
1523403
1722833
1955999
2155951
2422907
3089055
3825680
4688849
6557984
7222428
7288398
8189822
9921057
10853435
11753228
12652777
14353300
14951987
17051516
18017947
18818488
19985025
